In [1]:
import pandas as pd
import numpy as np
import os
from datetime import datetime

NUMBER_OF_ZONES = 265

C:\Users\edsus\anaconda3\envs\mllabs\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [7]:
#get distance mat and df
intra_zone_matrix = np.load("intra_zone_matrix.npy")
df = pd.read_parquet("all_cleaned_data/all_cleaned_data.parquet", engine='fastparquet')
df.head()

,VendorID,PickupDatetime,DropoffDatetime,TripDuration,PassengerCount,TripDistance,PULocationID,DOLocationID,PaymentType,FareAmount,ExtraCharges,MTATax,TipAmount,TollsAmount,ImprovementSurcharge,TotalAmount,CongestionSurcharge,AirportFee
0,2,2024-01-01 00:46:55,2024-01-01 00:58:25,11.500000,1.0,1.98,236,239,1.0,12.8,1.0,0.5,3.61,0.0,1.0,21.66,2.75,0
1,2,2024-01-01 00:31:42,2024-01-01 00:52:34,20.866667,5.0,6.54,65,170,1.0,30.3,1.0,0.5,7.11,0.0,1.0,42.66,2.75,0
2,2,2024-01-01 00:30:21,2024-01-01 00:49:23,19.033333,1.0,3.08,74,262,1.0,19.8,1.0,0.5,3.00,0.0,1.0,28.05,2.75,0
3,1,2024-01-01 00:30:20,2024-01-01 00:42:12,11.866667,1.0,2.40,74,116,2.0,14.2,1.0,1.5,0.00,0.0,1.0,16.70,0.00,0
4,2,2024-01-01 00:32:38,2024-01-01 00:43:37,10.983333,1.0,5.14,74,243,1.0,22.6,1.0,0.5,6.28,0.0,1.0,31.38,0.00,0


In [ ]:
#define functions

#Find all close zones for each zone
def close_zones_fn(intra_zone_matrix):
    close_zones = np.empty(266, dtype=object)
    for p in range(266):
        close_zones[p] = []
        for d in range(266):
            if intra_zone_matrix[p][d][0] < 15:
                close_zones[p].append(d)
                
    return close_zones
         
#generate the probability transition matrix
def probability_transition_matrix(df, intra_zone_matrix):
    
    #find all the close zones
    close_zones = close_zones_fn(intra_zone_matrix)
    
    #initialise matrix with shape: (hour of the day, dropoff zone, next likely pickup zone)
    ptm = np.zeros((24,266,266))
    
    #this is only the first 500- if this function seems accurate we can run it on the whole data on a labmachine/desktop 
    for i, row in df.head(500).iterrows():

        #Find the Hour and the Pickup Zone
        hour = row["PickupDatetime"].hour
        pu_zone = row["PULocationID"]

        # For every zone that is close, we +1 in its relevant slot that this pickup was made, thereby increasing its associate demand
        # Essentially, we accumulate all the pickups that have been made nearby for each zone in each hour.
        for close_zone in close_zones:
            ptm[hour, close_zone, pu_zone]+=1

    row_sums = ptm.sum(axis=2, keepdims=True)

    # normalise but avoid division by zero
    ptm_unshaped = np.divide(ptm, row_sums, where=row_sums != 0)
    
    #row_sums_check = ptm.sum(axis=2) #Veify for sanity that all add to 1
    
    # reshape matrix to shape (24 * 266, 266)
    ptm = ptm_unshaped.reshape(-1, ptm_unshaped.shape[2])
    
    return ptm_unshaped, ptm




In [14]:
# save matrix
filename = "probability_transition_matrix.npy"
filename_unshaped = "probability_transition_matrix_unshaped.npy"

ptm, ptm_unshaped = probability_transition_matrix(df, intra_zone_matrix)
np.save(filename, ptm)
np.save(filename_unshaped, ptm_unshaped

print("Probability Transition Matrix saved at ", filename)
print("Unshaped Probability Transition Matrix saved at ", filename_unshaped)

Probability Transition Matrix saved at  probability_transition_matrix.npy
